In [1]:
import os
import numpy as np
import pandas as pd
from ieeg.auth import Session

from get_iEEG_data import *
from spike_detector import *
from iEEG_helper_functions import *

SPIKES_OUTPUT_DIR = "../../Data/spikes/devin_spikes/"

In [2]:
def create_pwd_file(username, password, fname=None):
    if fname is None:
        fname = "{}_ieeglogin.bin".format(username[:3])
    with open(fname, "wb") as f:
        f.write(password.encode())
    print("-- -- IEEG password file saved -- --")


create_pwd_file("dma", "mycqEv-pevfo4-roqfan")

with open("dma_ieeglogin.bin", "r") as f:
    session = Session("dma", f.read())

-- -- IEEG password file saved -- --


In [3]:
dataset_name = "HUP143_phaseII"
dataset = session.open_dataset(dataset_name)

In [4]:
all_channel_labels = np.array(dataset.get_channel_labels())
label_idxs = electrode_selection(all_channel_labels)
labels = all_channel_labels[label_idxs]

In [5]:
duration_usec = dataset.get_time_series_details(labels[5]).duration
duration_hours = int(duration_usec / 1000000 / 60 / 60)
enlarged_duration_hours = duration_hours + 24

In [6]:
print(f"Opening {dataset_name} with duration {duration_hours} hours")

# Calculate the total number of 2-minute intervals in the enlarged duration
total_intervals = enlarged_duration_hours * 30  # 60min/hour / 2min = 30

# Loop through each 2-minute interval
for interval in range(4652, total_intervals):
    print(f"Getting iEEG data for interval {interval} out of {total_intervals}")
    duration_usec = 1.2e8  # 2 minutes
    start_time_usec = interval * 2 * 60 * 1e6  # 2 minutes in microseconds
    stop_time_usec = start_time_usec + duration_usec

    try:
        ieeg_data, fs = get_iEEG_data(
            "dma",
            "dma_ieeglogin.bin",
            dataset_name,
            start_time_usec,
            stop_time_usec,
            labels,
        )
    except Exception as e:
        # handle the exception
        print(f"Error: {e}")
        break

    # Drop rows that has any nan
    ieeg_data = ieeg_data.dropna(axis=0, how="any")
    if ieeg_data.empty:
        print("Empty dataframe, skip saving...")
        continue

    fs = int(fs)

    good_channels_res = detect_bad_channels_optimized(ieeg_data.to_numpy(), fs)
    good_channel_indicies = good_channels_res[0]
    good_labels = labels[good_channel_indicies]
    ieeg_data = ieeg_data[good_labels]

    ieeg_data = common_average_montage(ieeg_data)

    # Apply the filters directly on the DataFrame
    ieeg_data = pd.DataFrame(notch_filter(ieeg_data.values, 59, 61, fs))
    ieeg_data = pd.DataFrame(bandpass_filter(ieeg_data.values, 1, 70, fs))

    spike_output = spike_detector(
        data=ieeg_data.to_numpy(),
        fs=fs,
        labels=good_labels,
    )
    if len(spike_output) == 0:
        print("No spikes detected, skip saving...")
        continue
    else:
        print(f"{len(np.unique(spike_output[:, 2]))} spikes detected")

    # Save the spike_output to a npy file
    np.save(
        os.path.join(SPIKES_OUTPUT_DIR, f"{dataset_name}_{interval}.npy"),
        spike_output,
    )

Opening HUP143_phaseII with duration 239 hours
Getting iEEG data for interval 4652 out of 7890
Using optimized spike detector...
'the big loop' took 1.1533 seconds to execute.
'multi_channel_requirement' took 0.0015 seconds to execute.
28 spikes detected
Getting iEEG data for interval 4653 out of 7890
Using optimized spike detector...
'the big loop' took 0.8289 seconds to execute.
'multi_channel_requirement' took 0.0003 seconds to execute.
3 spikes detected
Getting iEEG data for interval 4654 out of 7890
Using optimized spike detector...
'the big loop' took 0.8910 seconds to execute.
'multi_channel_requirement' took 0.0003 seconds to execute.
5 spikes detected
Getting iEEG data for interval 4655 out of 7890
Using optimized spike detector...
'the big loop' took 0.8606 seconds to execute.
'multi_channel_requirement' took 0.0002 seconds to execute.
2 spikes detected
Getting iEEG data for interval 4656 out of 7890
Using optimized spike detector...
'the big loop' took 0.8513 seconds to exec